In [1]:
import riskfolio as rp 
import pandas as pd
import numpy as np
from openbb_terminal.sdk import openbb
import datetime
from dateutil.relativedelta import relativedelta
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe
import yfinance as yf
import bt 
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/adamjohnson/Documents/GoogleCloudService/gspread-api-394113-fcd586f615d5.json', scope)

client = gspread.authorize(credentials)

#sheet =client.open('Risk Parity Database')

sheet =client.open_by_key('1_XjLk6Vrz7ht5twTK_jARrNf8pBkgVztr8ft5o8ADow')

sheet_instance = sheet.get_worksheet(0)

data = sheet_instance.get_all_values()

portfolio = pd.DataFrame(data)

headers = portfolio.iloc[0]
portfolio = pd.DataFrame(portfolio.values[1:], columns=headers)

portfolio.set_index('Ticker', inplace=True)

portfolio = portfolio[['CVaR','CVaR Allocation']]

portfolio['CVaR'] = portfolio['CVaR'].astype(float)
portfolio['CVaR Allocation'] = portfolio['CVaR Allocation'].astype(float)

portfolio = portfolio[['CVaR','CVaR Allocation']]


portfolio.reset_index(inplace=True)
portfolio.rename(columns={'index': 'Ticker'}, inplace=True)

portfolio

,Ticker,CVaR,CVaR Allocation
0,CEG,0.071,1376.925306
1,PANW,0.039,758.375903
2,AAPL,0.080,1554.042536
3,NOW,0.037,713.687062
4,NVDA,0.046,897.310175
5,MSFT,0.081,1581.213576
6,AMZN,0.054,1041.427960
7,RTX,0.084,1638.514241
8,JPM,0.073,1413.750282
9,ULTA,0.095,1845.433037


In [7]:
portfolio_value = portfolio['CVaR Allocation'].sum()

end = pd.Timestamp(datetime.date.today())
#start = end - relativedelta(years=1)
start = '2023-11-16'

symbols = portfolio['Ticker'].tolist()

#stock_data = yf.download(symbols, start= start, end= end)['Adj Close']

stock_data = openbb.stocks.ca.hist(symbols,start, end)

#stock_data.dropna(how="any", axis=1, inplace=True)

stock_data.reset_index(inplace=True)
#stock_data.rename(columns={'index': 'Ticker'}, inplace=True)

stock_data



,Date,CEG,PANW,AAPL,NOW,NVDA,MSFT,AMZN,RTX,JPM,ULTA,FTNT,CVX,GEHC,LOW,CMI
0,2023-11-16,121.669998,242.300003,189.710007,652.280029,494.756500,376.170013,142.830002,80.230003,150.521835,405.549988,50.419998,141.770004,71.891701,201.267944,221.742844
1,2023-11-17,121.690002,247.589996,189.690002,654.359985,492.936676,369.850006,145.179993,79.680000,151.883438,409.269989,51.259998,144.460007,72.971275,202.680847,223.807236
2,2023-11-20,122.279999,260.570007,191.449997,666.919983,504.045685,377.440002,146.130005,79.169998,152.350555,405.609985,51.910000,144.380005,73.301147,203.417145,222.477295
3,2023-11-21,122.510002,264.029999,190.639999,668.400024,499.396088,373.070007,143.899994,78.150002,152.032516,404.920013,52.580002,144.039993,73.351128,197.069061,222.119995
4,2023-11-22,123.250000,263.140015,191.309998,669.659973,487.117188,377.850006,146.710007,79.820000,152.390305,409.489990,52.970001,144.330002,73.571037,197.486954,224.509995
5,2023-11-24,123.860001,266.079987,189.970001,673.900024,477.718018,377.429993,146.740005,79.980003,152.599014,414.720001,53.430000,144.899994,73.681000,198.074020,225.490005
6,2023-11-27,125.099998,269.089996,189.789993,670.549988,482.377594,378.609985,147.729996,79.470001,152.251175,414.750000,53.740002,144.360001,71.112007,198.820267,224.410004
7,2023-11-28,124.449997,277.829987,190.399994,665.239990,478.167938,382.700012,147.029999,80.190002,152.599014,417.959991,52.790001,145.509995,68.103195,199.049103,223.750000
8,2023-11-29,120.580002,288.910004,189.369995,678.929993,481.357666,378.850006,146.320007,80.949997,153.374252,420.010010,53.599998,143.910004,67.683357,198.919754,223.949997
9,2023-11-30,121.040001,295.089996,189.949997,685.739990,467.658905,378.910004,146.089996,81.480003,155.123459,425.989990,52.560001,143.600006,68.433060,197.835220,224.160004


In [8]:
results = pd.DataFrame(columns=['Date', 'Cumulative_Portfolio_Value'])

# Assuming that your price_data DataFrame has a 'Date' column, include it in the results DataFrame.
results['Date'] = stock_data['Date']

# Initialize the first row with starting values
results.at[0, 'Cumulative_Portfolio_Value'] = portfolio_value
for ticker in portfolio['Ticker']:
    allocation = portfolio.loc[portfolio['Ticker'] == ticker, 'CVaR'].values[0]
    results.at[0, f'{ticker}_Contribution'] = portfolio_value * allocation

# Iterate through stock tickers and calculate portfolio value and contributions
for ticker in portfolio['Ticker']:
    returns_column = f'{ticker}_Return'
    allocation = portfolio.loc[portfolio['Ticker'] == ticker, 'CVaR'].values[0]
    for i in range(1, len(results)):
        results.at[i, f'{ticker}_Return'] = stock_data.at[i, ticker] / stock_data.at[i - 1, ticker] - 1
        results.at[i, f'{ticker}_Contribution'] = results.at[i - 1, f'{ticker}_Contribution'] * (1 + results.at[i, f'{ticker}_Return'])

# Calculate daily portfolio value and cumulative portfolio value
for i in range(1, len(results)):
    cumulative_portfolio_value = portfolio_value
    for ticker in portfolio['Ticker']:
        contribution_column = f'{ticker}_Contribution'
        cumulative_portfolio_value += results.at[i, contribution_column]
    results.at[i, 'Cumulative_Portfolio_Value'] = cumulative_portfolio_value


results

,Date,Cumulative_Portfolio_Value,CEG_Contribution,PANW_Contribution,AAPL_Contribution,NOW_Contribution,NVDA_Contribution,MSFT_Contribution,AMZN_Contribution,RTX_Contribution,...,MSFT_Return,AMZN_Return,RTX_Return,JPM_Return,ULTA_Return,FTNT_Return,CVX_Return,GEHC_Return,LOW_Return,CMI_Return
0,2023-11-16,19415.0,1378.465000,757.185000,1553.200000,718.355000,893.090000,1572.615000,1048.410000,1630.860000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-11-17,38942.542821,1378.691639,773.716174,1553.036220,720.645652,889.805017,1546.193600,1065.659554,1619.679918,...,-0.016801,0.016453,-0.006855,0.009046,0.009173,0.016660,0.018974,0.015017,0.007020,0.009310
2,2023-11-20,39066.485536,1385.376026,814.278574,1567.445705,734.477959,909.858002,1577.924311,1072.632895,1609.312948,...,0.020522,0.006544,-0.006401,0.003075,-0.008943,0.012680,-0.000554,0.004521,0.003633,-0.005942
3,2023-11-21,38971.067243,1387.981858,825.091012,1560.814067,736.107926,901.464966,1559.655126,1056.264025,1588.579162,...,-0.011578,-0.015260,-0.012884,-0.002088,-0.001701,0.012907,-0.002355,0.000682,-0.031207,-0.001606
4,2023-11-22,39093.410905,1396.365692,822.309820,1566.299498,737.495506,879.300199,1579.638371,1076.890262,1622.525729,...,0.012813,0.019528,0.021369,0.002353,0.011286,0.007417,0.002013,0.002998,0.002121,0.010760
5,2023-11-24,39140.42314,1403.276718,831.497202,1555.328636,742.165068,862.333661,1577.882465,1077.110461,1625.778173,...,-0.001112,0.000204,0.002005,0.001370,0.012772,0.008684,0.003949,0.001495,0.002973,0.004365
6,2023-11-27,39112.752136,1417.325322,840.903452,1553.854869,738.475677,870.744710,1582.815538,1084.377252,1615.411202,...,0.003126,0.006747,-0.006377,-0.002279,0.000072,0.005802,-0.003727,-0.034866,0.003768,-0.004790
7,2023-11-28,39102.928269,1409.961105,868.215830,1558.849086,732.627785,863.145818,1599.914289,1079.239089,1630.046889,...,0.010803,-0.004738,0.009060,0.002285,0.007740,-0.017678,0.007966,-0.042311,0.001151,-0.002941
8,2023-11-29,39106.21555,1366.115844,902.840769,1550.416246,747.704564,868.903628,1583.818968,1074.027564,1645.495532,...,-0.010060,-0.004829,0.009477,0.005080,0.004905,0.015344,-0.010996,-0.006165,-0.000650,0.000894
9,2023-11-30,39159.056186,1371.327421,922.153182,1555.164856,755.204404,844.175855,1584.069793,1072.339222,1656.269135,...,0.000158,-0.001572,0.006547,0.011405,0.014238,-0.019403,-0.002154,0.011077,-0.005452,0.000938


In [9]:
# Step 1: Select columns containing the word 'Contribution'
contribution_columns = [col for col in results.columns if 'Contribution' in col]

# Step 2: Create a new column that sums each row of 'Contribution' columns
results['Total_Contribution'] = results[contribution_columns].sum(axis=1)

results

,Date,Cumulative_Portfolio_Value,CEG_Contribution,PANW_Contribution,AAPL_Contribution,NOW_Contribution,NVDA_Contribution,MSFT_Contribution,AMZN_Contribution,RTX_Contribution,...,AMZN_Return,RTX_Return,JPM_Return,ULTA_Return,FTNT_Return,CVX_Return,GEHC_Return,LOW_Return,CMI_Return,Total_Contribution
0,2023-11-16,19415.0,1378.465000,757.185000,1553.200000,718.355000,893.090000,1572.615000,1048.410000,1630.860000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19415.000000
1,2023-11-17,38942.542821,1378.691639,773.716174,1553.036220,720.645652,889.805017,1546.193600,1065.659554,1619.679918,...,0.016453,-0.006855,0.009046,0.009173,0.016660,0.018974,0.015017,0.007020,0.009310,19527.542822
2,2023-11-20,39066.485536,1385.376026,814.278574,1567.445705,734.477959,909.858002,1577.924311,1072.632895,1609.312948,...,0.006544,-0.006401,0.003075,-0.008943,0.012680,-0.000554,0.004521,0.003633,-0.005942,19651.485536
3,2023-11-21,38971.067243,1387.981858,825.091012,1560.814067,736.107926,901.464966,1559.655126,1056.264025,1588.579162,...,-0.015260,-0.012884,-0.002088,-0.001701,0.012907,-0.002355,0.000682,-0.031207,-0.001606,19556.067243
4,2023-11-22,39093.410905,1396.365692,822.309820,1566.299498,737.495506,879.300199,1579.638371,1076.890262,1622.525729,...,0.019528,0.021369,0.002353,0.011286,0.007417,0.002013,0.002998,0.002121,0.010760,19678.410905
5,2023-11-24,39140.42314,1403.276718,831.497202,1555.328636,742.165068,862.333661,1577.882465,1077.110461,1625.778173,...,0.000204,0.002005,0.001370,0.012772,0.008684,0.003949,0.001495,0.002973,0.004365,19725.423140
6,2023-11-27,39112.752136,1417.325322,840.903452,1553.854869,738.475677,870.744710,1582.815538,1084.377252,1615.411202,...,0.006747,-0.006377,-0.002279,0.000072,0.005802,-0.003727,-0.034866,0.003768,-0.004790,19697.752136
7,2023-11-28,39102.928269,1409.961105,868.215830,1558.849086,732.627785,863.145818,1599.914289,1079.239089,1630.046889,...,-0.004738,0.009060,0.002285,0.007740,-0.017678,0.007966,-0.042311,0.001151,-0.002941,19687.928269
8,2023-11-29,39106.21555,1366.115844,902.840769,1550.416246,747.704564,868.903628,1583.818968,1074.027564,1645.495532,...,-0.004829,0.009477,0.005080,0.004905,0.015344,-0.010996,-0.006165,-0.000650,0.000894,19691.215550
9,2023-11-30,39159.056186,1371.327421,922.153182,1555.164856,755.204404,844.175855,1584.069793,1072.339222,1656.269135,...,-0.001572,0.006547,0.011405,0.014238,-0.019403,-0.002154,0.011077,-0.005452,0.000938,19744.056186


In [10]:
# Calculate the percentage gain/loss relative to the first row
results['Percentage_Gain_Loss'] = ((results['Total_Contribution'] - results['Total_Contribution'].iloc[0]) / results['Total_Contribution'].iloc[0])

results 

,Date,Cumulative_Portfolio_Value,CEG_Contribution,PANW_Contribution,AAPL_Contribution,NOW_Contribution,NVDA_Contribution,MSFT_Contribution,AMZN_Contribution,RTX_Contribution,...,RTX_Return,JPM_Return,ULTA_Return,FTNT_Return,CVX_Return,GEHC_Return,LOW_Return,CMI_Return,Total_Contribution,Percentage_Gain_Loss
0,2023-11-16,19415.0,1378.465000,757.185000,1553.200000,718.355000,893.090000,1572.615000,1048.410000,1630.860000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19415.000000,0.000000
1,2023-11-17,38942.542821,1378.691639,773.716174,1553.036220,720.645652,889.805017,1546.193600,1065.659554,1619.679918,...,-0.006855,0.009046,0.009173,0.016660,0.018974,0.015017,0.007020,0.009310,19527.542822,0.005797
2,2023-11-20,39066.485536,1385.376026,814.278574,1567.445705,734.477959,909.858002,1577.924311,1072.632895,1609.312948,...,-0.006401,0.003075,-0.008943,0.012680,-0.000554,0.004521,0.003633,-0.005942,19651.485536,0.012181
3,2023-11-21,38971.067243,1387.981858,825.091012,1560.814067,736.107926,901.464966,1559.655126,1056.264025,1588.579162,...,-0.012884,-0.002088,-0.001701,0.012907,-0.002355,0.000682,-0.031207,-0.001606,19556.067243,0.007266
4,2023-11-22,39093.410905,1396.365692,822.309820,1566.299498,737.495506,879.300199,1579.638371,1076.890262,1622.525729,...,0.021369,0.002353,0.011286,0.007417,0.002013,0.002998,0.002121,0.010760,19678.410905,0.013567
5,2023-11-24,39140.42314,1403.276718,831.497202,1555.328636,742.165068,862.333661,1577.882465,1077.110461,1625.778173,...,0.002005,0.001370,0.012772,0.008684,0.003949,0.001495,0.002973,0.004365,19725.423140,0.015989
6,2023-11-27,39112.752136,1417.325322,840.903452,1553.854869,738.475677,870.744710,1582.815538,1084.377252,1615.411202,...,-0.006377,-0.002279,0.000072,0.005802,-0.003727,-0.034866,0.003768,-0.004790,19697.752136,0.014564
7,2023-11-28,39102.928269,1409.961105,868.215830,1558.849086,732.627785,863.145818,1599.914289,1079.239089,1630.046889,...,0.009060,0.002285,0.007740,-0.017678,0.007966,-0.042311,0.001151,-0.002941,19687.928269,0.014058
8,2023-11-29,39106.21555,1366.115844,902.840769,1550.416246,747.704564,868.903628,1583.818968,1074.027564,1645.495532,...,0.009477,0.005080,0.004905,0.015344,-0.010996,-0.006165,-0.000650,0.000894,19691.215550,0.014227
9,2023-11-30,39159.056186,1371.327421,922.153182,1555.164856,755.204404,844.175855,1584.069793,1072.339222,1656.269135,...,0.006547,0.011405,0.014238,-0.019403,-0.002154,0.011077,-0.005452,0.000938,19744.056186,0.016949


In [11]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/adamjohnson/Documents/GoogleCloudService/gspread-api-394113-fcd586f615d5.json', scope)

client = gspread.authorize(credentials)

#sheet =client.open('Risk Parity Database')

sheet =client.open_by_key('1_XjLk6Vrz7ht5twTK_jARrNf8pBkgVztr8ft5o8ADow')

sheet_instance = sheet.get_worksheet(4)

# set dataframe for initial write the use the appending code below for next iterations 
set_with_dataframe(sheet_instance,results)